## Basic Prompt UI

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
%pip install gradio streamlit

In [ ]:
from langchain_anthropic import ChatAnthropic
model = ChatAnthropic(model='claude-3-haiku-20240307')

import streamlit as st
user_input = st.text_input('Enter your prompt')
if st.button('Summarize Reseach Paper'):
    result = model.invoke(user_input)   # result = model.invoke('Hello world')
    st.write(result.content)            # print(result.content)

## Dynamic prompt - using PromptTemplate

In [ ]:
# why not fstring
# 1. code validation
# 2. re-use by save-load in json format
# 3. langchain ecosystem

In [ ]:
from langchain_core.prompts import PromptTemplate, load_prompt
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name='gpt-3.5-turbo')

product1 = input()   # or choose from List by steamlit dropdown
template = PromptTemplate(template="""
I want you to act as a naming consultant for new companies.
What is a good name for a company that makes {product}?""", input_variables=['product'], validate_template= True)

prompt = template.invoke({'product':product1}) # if input_variables=[] here send empty dict {}
result = model.invoke(prompt)
print(result.content)

In [ ]:
# OR
chain = template | model
result = chain.invoke({'product':product1})
print(result.content)

In [ ]:
template.save('templates/01_template_for_name.json')
# loaded_ptemplate = load_prompt('templates/01_template_for_name.json') # load by

## Create chatbot - terminal based

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name='gpt-3.5-turbo')

In [ ]:
while True:
    user_input = input("You: ")
    if user_input == 'quit':
        break
    result = model.invoke(user_input)
    print("AI:", result.content)

In [ ]:
# the above code is all okay but 
# 1. can't remember early conversations (and who told what)
# 2. no template used

In [ ]:
chat_history = []

while True:
    user_input = input("You: ")
    chat_history.append(user_input)
    if user_input == 'quit':
        break
    result = model.invoke(chat_history) # not only user_input
    chat_history.append(result.content)
    print("AI: ", result.content)

# print("ENTIRE CHAT STORED:")
# print(*chat_history)

In [ ]:
# above we easily cant differentiate which one is AI msg or USR msg, so use `messages`

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

model= ChatOpenAI(model_name='gpt-3.5-turbo')
chat_history = [SystemMessage(content="You are a doctor AI bot.")] # FIXED system message
                                    # then append Human/AI msg accordingly

while True:
    user_input = input("You: ")
    chat_history.append(HumanMessage(content=user_input)) # no template used
    if user_input == 'quit':
        break
    result = model.invoke(chat_history) # NB
    chat_history.append(AIMessage(content=result.content))
    print(result.content)

# print("ENTIRE CHAT STORED:")
# print(*chat_history)

## Template for reusable, parameterized initial prompts.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

# Using ChatPromptTemplate([...]) instead of ChatPromptTemplate.from_messages according to latest docs
chat_template = ChatPromptTemplate([
    ("system", "You are a helpful {domain} AI bot."),
    ("human", "explain in simple terms about {topic}"),
                                                            # how AIMessage? => "ai"
])

prompt = chat_template.invoke({'domain':'medical', 'topic':'type2 diabetes'})
print(prompt)

# then the entire code to get content

## MessagesPlaceholder
Loading older chats from db/txt/blob_storage to ChatPromptTemplate
### Full fleged-use of templates

In [ ]:
# creating synthetic txt file as history

with open('04_chat_history.txt', 'w') as f:
    f.write('HumanMessage(content="I want to request a refund for my order #12345.")\n')
    f.write('AIMessage(content="Your refund request for order #12345 has been initiated. It will be processed in 3-5 business days.")')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# chat template
chat_template = ChatPromptTemplate([
    ("system", "You are a helpful customer support agent."),
    MessagesPlaceholder(variable_name="chat_history"), # here history is shared
    ("human", "{query}")
])

# load chat history
chat_history = []
with open('04_chat_history.txt', 'r') as f:
    # chat_history.extend(f.readlines()) # naive way, not recommended
    # below code is verbose and manual work needed because of not-so-mature 🦜🔗 ecosystem
    for line in f:
        if 'HumanMessage' in line:
            content = line.split('content="')[1].split('"')[0]
            chat_history.append(HumanMessage(content=content))
        elif 'AIMessage' in line:
            content = line.split('content="')[1].split('"')[0]
            chat_history.append(AIMessage(content=content))
# print(chat_history)

# create prompt
prompt = chat_template.invoke({'chat_history':chat_history, 'query':'where is my order?'})
print(prompt)

# the next as is
result = model.invoke(prompt)
print(result.text) # we can store these too

### Using gradio to showcase

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
import gradio as gr
import os

# Init LLM
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

# Chat template
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful customer support agent."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{query}")
])

# Load chat history (optional)
chat_history = []
if os.path.exists('04_chat_history.txt'):
    with open('04_chat_history.txt', 'r') as f:
        for line in f:
            if 'HumanMessage' in line:
                content = line.split('content="')[1].split('"')[0]
                chat_history.append(HumanMessage(content=content))
            elif 'AIMessage' in line:
                content = line.split('content="')[1].split('"')[0]
                chat_history.append(AIMessage(content=content))

# Chat function
def chat_fn(query, history_ui):
    global chat_history

    # Build prompt using LangChain
    prompt = chat_template.invoke({
        'chat_history': chat_history,
        'query': query
    })

    # Get response from OpenAI
    response = model.invoke(prompt).content

    # Update in-memory history
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response))

    # Save to file
    with open("04_chat_history.txt", "a", encoding="utf-8") as f:
        f.write(f'HumanMessage content="{query}"\n')
        f.write(f'AIMessage content="{response}"\n')

    return response  # Gradio expects a string here

# Gradio UI
gr.ChatInterface(
    fn=chat_fn,
    chatbot=gr.Chatbot(),
    textbox=gr.Textbox(placeholder="Ask your question...", show_label=False),
    title="Customer Support Chatbot"
).launch()


In [ ]:
## more prompting techniques - few shot template, chain of thought, gk